In [1]:
# import warnings
# warnings.simplefilter('ignore')
# warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')


import spacy
# gpu = spacy.prefer_gpu()
# print('GPU:', gpu)
# pip install -U spacy[cuda100]
# python -m spacy validate
# python -m spacy download en_core_web_sm

from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint

nlp = en_core_web_sm.load()
import re
from stopwords import *
import nltk
from preprocess import *
# from feature import *

from textblob import TextBlob
import collections

from spacy.symbols import cop, acomp, amod, conj, neg, nn, nsubj, dobj,prep,advmod
from spacy.symbols import VERB, NOUN, PROPN, ADJ, ADV, AUX, PART

pattern_counter = collections.Counter()

from nltk.corpus import stopwords
import networkx as nx
from MongoDB import MongoDB
import os
import pickle
import warnings
from tqdm import tqdm
from collections import Counter,OrderedDict

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# 斷詞辭典

In [2]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
stpwords_list3 = [f.replace("\n","") for f in open("stopwords.txt","r",encoding = "utf-8").readlines()]
stpwords_list3.remove("not")
stopwords = list(html_escape_table + stpwords_list2) + list(list(stops) + list(stpwords_list1) + list(stpwords_list3))

# Total Opinion
# https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#datasets
<!-- Opinion Lexicon: A list of English positive and negative opinion words or sentiment words (around 6800 words). This list was compiled over many years starting from our first paper (Hu and Liu, KDD-2004). -->

In [3]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
# opinion_lexicon["total-words"]

negative-words.txt
positive-words.txt


# pickle 提取

In [4]:
# reload a file to a variable
from glob import glob
with open(glob('new_*.pickle')[0], 'rb') as file:
    print(file.name)
    PROD_DICT =pickle.load(file)

new_DVD Players.pickle


# 句子篩選/切割

In [5]:
import textacy
from summa import keywords,summarizer

def sentence_extract_blob(text):    
    extract_sents = []
    text = text.replace(",","").replace("i.e.","")
    text = SentProcess(text)
    
    for sent in text.split("<end>"):        
#         print(sent)
        if sent == "": continue
        sentiment = TextBlob(sent).sentences[0].sentiment
#         if abs(sentiment.polarity) <= 0.5: continue
#         if abs(sentiment.subjectivity) <= 0.5: continue
#         print(TextBlob(sent).sentences)
#         print("polarity : ",sentiment.polarity)
#         print("subjectivity : ",sentiment.subjectivity)
# #         print(keywords.keywords(sent))
#         print("***")
        extract_sents.append(sent)
    return extract_sents

def extract_cand_pharse(text): 
    sent_pattern = r"""( 
    (<NOUN><NOUN> | <ADJ><NOUN> | <NOUN><NOUN><NOUN> | <NOUN><ADP><DET><NOUN> | 
    <NOUN> | <ADJ><NOUN><NOUN> | <ADJ><ADJ><NOUN> | <NOUN><PREP><NOUN> | <NOUN><PREP><DET><NOUN> | 
    <VERB><PUNCT><ADP><NOUN><NOUN> | <DET><NOUN> )


    (<AUX><ADV><ADJ>|<AUX><ADJ>|<ADV><ADJ>|<ADJ><NOUN>) 
    )
    """
#     pattern = r'<PROPN>+ (<PUNCT|CCONJ> <PUNCT|CCONJ>? <PROPN>+)*'
    extract_pharse = []
    doc = textacy.make_spacy_doc(text,lang='en_core_web_sm')
    phrases = textacy.extract.pos_regex_matches(doc, sent_pattern)
    # Print all Verb Phrase

    for phrase in phrases:
#         print("verb_phrases : " , phrase.text)
    #     print([(token.text,token.pos_) for token in nlp(chunk.text)])
        extract_pharse.append(phrase.text+"\n")           
    return extract_pharse        


# 規格關鍵字提取

In [6]:
def getKeywordFeat(description,feature):  # 搜索關鍵保留字，並重新清理句子       
    description_list = []
    for sent in description:
        if len(TextBlob(sent)) > 1:
            sent_list = sent.split(".")
            description_list.extend(sent_list) 
        else:
            description_list.extend(sent) 
    description_list = [sent.strip() +"." for sent in description_list]
    if type(feature) == str:
        total_desc_sents = description_list + [feature]
    else:
        total_desc_sents = description_list + feature    
 
    # Stage 1 取dash feature
    keywords = []
    newdescription = ""
    for sent in total_desc_sents:
        for token in nlp(sent):
            sent = sent.lower()
            for k, v in contractions.items():
                if k in sent:
                    sent = sent.replace(k, v)

            for k in html_escape_table:
                if k in sent:
                    sent = sent.replace(k, "")
            
            
            pattern = re.compile(r"([\d\w\.-]+[-'//.][\d\w\.-]+)")  # |  ([\(](\w)+[\)])
            keys = pattern.findall(sent)
            
            
        keywords.extend(keys)
        cleansent = sent
        for k in keys:    
            k = k.replace("(","").replace(")","")
            cleansent = cleansent.replace(k,"(" + k + ")")
            cleansent = cleansent.replace("((","(").replace("))",")")     
            cleansent = cleansent.replace("(" + k + ")","") # 移除符號特徵
            cleansent = cleansent.strip()
        
#         cleansent = cleansent + ".\n"
        cleansent = remove_word3(cleansent)
        char = " "
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)  
        char = "."
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)            
         
        if len(cleansent) < 5 : continue
        newdescription = newdescription + cleansent + ".\n"
        
    newdescription = newdescription.replace("..",".")   
    newdescription = re.sub(r"\([\w]+\)","",newdescription)
    newdescription = re.sub(r"\(\)","",newdescription)   
    newdescription = re.sub(r'\"',"",newdescription)
    
    # get Noun pharse keyword for newdescription
#     for pharse in nlp(newdescription).noun_chunks:
#         print(pharse.text)
    cand_pf = PF_rule_POS(newdescription).run()
#     print(cand_pf)
    
    keywords2 = set()
    for pf in cand_pf:
        chunk_pfs = nltk.word_tokenize(pf)
        for tok in chunk_pfs:
            if tok in stopwords: continue
            if tok in opinion_lexicon["total-words"]: continue
            keywords2.add(tok)

#     print(len(keywords2),keywords2)
    return keywords,keywords2, newdescription

# 評論濾除符號-英文特徵

In [7]:
def cleanReview(review):  # 搜索關鍵保留字，並重新清理句子     
    # Stage 1 取dash feature
    keywords = []
    newReview = ""
    for sent in nltk.sent_tokenize(review):
        for token in nlp(sent):
            sent = sent.lower()
            for k, v in contractions.items():
                if k in sent:
                    sent = sent.replace(k, v)

            for k in html_escape_table:
                if k in sent:
                    sent = sent.replace(k, "")
            
            
            pattern = re.compile(r"([\d\w\.-]+[-'//.][\d\w\.-]+)")  # |  ([\(](\w)+[\)])
            keys = pattern.findall(sent)
            
            
        keywords.extend(keys)
        cleansent = sent
        for k in keys:    
            k = k.replace("(","").replace(")","")
            cleansent = cleansent.replace(k,"(" + k + ")")
            cleansent = cleansent.replace("((","(").replace("))",")")     
            cleansent = cleansent.replace("(" + k + ")","") # 移除符號特徵
            cleansent = cleansent.strip()
        
#         cleansent = cleansent + ".\n"
        cleansent = remove_word3(cleansent)
        char = " "
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)  
        char = "."
        while char * 2 in cleansent:
            cleansent = cleansent.replace(char * 2, char)            
         
        if len(cleansent) < 5 : continue
        newReview = newReview + cleansent + ".\n"
        
    newReview = newReview.replace("..",".")   
    newReview = re.sub(r"\([\w]+\)","",newReview)
    newReview = re.sub(r"\(\)","",newReview)   
    newReview = re.sub(r'\"',"",newReview)
    
    # get Noun pharse keyword for newdescription
#     cand_pf = PF_rule_POS(newReview).run()
    
#     keywords2 = set()
#     for pf in cand_pf:
#         chunk_pfs = nltk.word_tokenize(pf)
#         for tok in chunk_pfs:
#             if tok in stopwords: continue
#             if tok in opinion_lexicon["total-words"]: continue
#             keywords2.add(tok)

#     print(len(keywords2),keywords2)
    return keywords, newReview

# PF-Extraction-Rule(POS)

In [8]:
# -NN, 
# -NN NN, 
# JJ NN 
# -NN NN NN, 
# JJ NN NN, 
# JJ JJ NN, 
# NN IN NN 
# -NN IN DT NN

class PF_rule_POS():
    def __init__(self, article):
        self.article = article
        self.matched_sents = []  # Collect data of matched sentences to be visualized

    def collect_sents(self, matcher, doc, i, matches):
        match_id, start, end = matches[i]
        span = doc[start:end]  # Matched span
        sent = span.sent  # Sentence containing matched span
        # Append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            "label": "MATCH",
        }]
        self.matched_sents.append({"text": sent.text, "ents": match_ents})

    def match_pattern(self, sent, flit_keyword=None):
        res = []
        #         ('Dolby', 'PROPN'), ('Digital', 'PROPN')
        matcher = Matcher(nlp.vocab)
        #         matcher.add("pf1", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf2", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         #         matcher.add("specn3", self.collect_sents, [{'POS': 'ADJ'}, {'POS': 'NOUN'}])  # add pattern
        #         matcher.add("pf3", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf4", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf5", self.collect_sents, [{'POS': 'ADJ'}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf6", self.collect_sents,
        #                     [{'POS': 'ADJ'}, {'POS': 'ADJ'}, {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         #         ('inches', 'NOUN'), ('(', 'PUNCT'), ('W', 'NOUN'), ('x', 'SYM'), ('H', 'NOUN'), ('x', 'SYM'), ('D', 'NOUN'), (')', 'PUNCT')
        #         matcher.add("pf6.1", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'SYM'},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}])  # inches (W x H x D)
        #         matcher.add("pf6.2", self.collect_sents,
        #                     [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': 'SYM'}, {'POS': {"IN": ['NOUN', 'PROPN']}}, {'IS_PUNCT': True}])  # inches (W x H)
        #         matcher.add("pf7", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'PREP'},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf8", self.collect_sents, [{'POS': {"IN": ['NOUN', 'PROPN']}}, {'POS': 'PREP'}, {'POS': 'DET'},
        #                                                    {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern
        #         matcher.add("pf9", self.collect_sents,
        #                     [{'POS': 'VERB'}, {'POS': 'PUNCT'}, {'POS': 'ADP'}, {'POS': {"IN": ['NOUN', 'PROPN']}},
        #                      {'POS': {"IN": ['NOUN', 'PROPN']}}])  # add pattern

        matcher.add("pf1", self.collect_sents, [{'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf2", self.collect_sents, [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf3", self.collect_sents, [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}])  # add pattern    
        matcher.add("pf4", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])  # add pattern
        matcher.add("pf5", self.collect_sents,
                    [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf6", self.collect_sents,
                    [{'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['JJ']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf7", self.collect_sents,
                    [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['IN']}}, {'TAG': {"IN": ['NN']}}])
        matcher.add("pf8", self.collect_sents, [{'TAG': {"IN": ['NN']}}, {'TAG': {"IN": ['IN']}},
                                                {'TAG': {"IN": ['DT']}}, {'TAG': {"IN": ['NN']}}])

        doc = nlp(sent)
        matches = matcher(doc)
        # Serve visualization of sentences containing match with displaCy
        # set manual=True to make displaCy render straight from a dictionary
        # (if you're not running the code within a Jupyer environment, you can
        # use displacy.serve instead)
        #         displacy.render(self.matched_sents, style="ent", manual=True)
        for match_id, start, end in matches:
            # Get the string representation
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]  # The matched span
            #             print(match_id, string_id, start, end, span.text)
            if flit_keyword:
                found = False
                spanStr = span.text
                for word in spanStr.split(" "):
                    if word in flit_keyword: found = True; break
                if found: res.append(span.text)
            else:
                res.append(span.text)
        return res

    def run(self):
        total_res = []
        for sent in nltk.sent_tokenize(self.article):
            sent = sent.lower()
            sent = sent.replace(",", " ").replace(":", " ").replace("(", " ").replace(")", " ").replace("multi-",
                                                                                                        "multiple ")
            ress = self.match_pattern(sent)
            total_res.extend(ress)

        total_res = list(set(total_res))
        #         print("final res",total_res)
        return total_res



# word2Vec

In [9]:
# 引入 word2vec
from gensim.models import word2vec
from glob import glob
import sys

import gensim
import torch
import torch.nn as nn
import torchsnooper
import os
import numpy as np

# 引入日志配置
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 引入数据集

In [10]:
sentences = []
# for idx,path in enumerate(raw_path):
#     text = open(path,"r",encoding="utf-8").readline()
#     sentences.append(str(text).split()) # 切分词汇
#     if (idx % 10000 ==0 )and (idx > 0): print(idx)

In [11]:
Products = len(PROD_DICT.items())
print("%s Products\n"%(Products)) 

with tqdm(total=Products) as pbar:
    for asin,DATA_DICT in PROD_DICT.items():
        REVIEW_ITEM_LIST = DATA_DICT['REVIEW_ITEM_LIST']
        title = DATA_DICT['title']
        description = DATA_DICT['description']
        feature = DATA_DICT['feature']      
        dash_keywords,noun_keywords, newdescription = getKeywordFeat(description,feature)
        for sent in nltk.sent_tokenize(newdescription):
            sent = sent.replace("." ,"")
            sentences.append(str(sent).split()) # 切分词汇
 
        for pair in REVIEW_ITEM_LIST:
            review = pair["review"]           
            overall = pair["vote"] 
            rev_dash_keywords, newreview = cleanReview(review)
            for sent in nltk.sent_tokenize(newreview):
                sent = sent.replace("." ,"")
                sentences.append(str(sent).split()) # 切分词汇            
#             extract_sents = sentence_extract_blob(review)
#             for rev_sent in extract_sents:
#                 rev_sent = rev_sent.replace("s.","")
#                 rev_sent = re.sub(r'(?:^| )\w(?:$| )', " ", rev_sent).strip() # remove single alphbet
#                 rev_sent = rev_sent.replace(" ve "," ").replace(" ha "," ")                  

#                 rev_sent,pos_sent = lemm_sent_process2(rev_sent, remove_stopwords=True, summary=False, mode="spacy", withdot=False)
# #                     _,pos_sent = get_pos_sequence(rev_sent,remove_stopwords=False,summary=False,mode = "spacy",withdot =True)
#                 if pos_sent == None: continue
#                 if len(pos_sent) == 0 : continue 
#                 tok_sent = set(nltk.word_tokenize(rev_sent)) - set(stopwords)
# #                     tok_sent = [token for token in nltk.word_tokenize(rev_sent) if token not in stopwords]
#                 freq_words.update(tok_sent)    
#                 _Sents.append(tok_sent)

#         feature_counter.update(newkeywords)
#         total_keywords = total_keywords + newkeywords
        pbar.update(1)
        pbar.set_description("Processing %s" % asin)
pbar.close()

485 Products



Processing B0145QN4F2: 100%|██████████| 485/485 [07:49<00:00,  2.01it/s]


In [12]:
main_cat = 'All Electronics'
big_categories = 'Electronics'
small_categories='DVD Players'
vocab_count = 50000

In [13]:
w2vec = word2vec.Word2Vec(sentences, size=300, min_count=1,max_vocab_size=None,iter=10,
                          sorted_vocab=1,max_final_vocab=vocab_count)

2019-12-11 00:31:38,723 : INFO : collecting all words and their counts
2019-12-11 00:31:38,726 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-11 00:31:38,776 : INFO : PROGRESS: at sentence #10000, processed 156680 words, keeping 7359 word types
2019-12-11 00:31:38,806 : INFO : PROGRESS: at sentence #20000, processed 314535 words, keeping 10630 word types
2019-12-11 00:31:38,837 : INFO : PROGRESS: at sentence #30000, processed 474388 words, keeping 12950 word types
2019-12-11 00:31:38,864 : INFO : PROGRESS: at sentence #40000, processed 622936 words, keeping 14444 word types
2019-12-11 00:31:38,869 : INFO : collected 14583 word types from a corpus of 633779 raw words and 40788 sentences
2019-12-11 00:31:38,877 : INFO : max_final_vocab=50000 and min_count=1 resulted in calc_min_count=1, effective_min_count=1
2019-12-11 00:31:38,879 : INFO : Loading a fresh vocabulary
2019-12-11 00:31:38,901 : INFO : effective_min_count=1 retains 14583 unique words (100

In [14]:
# write vocab to file
vocab_file = 'word2Vec/gensim_%s_%s.vocab'%(big_categories,small_categories)
if not os.path.isdir("word2Vec"):
    os.mkdir("word2Vec")
print("Writing vocab file...")
with open(vocab_file, 'w',encoding='utf-8') as writer:
    for word in w2vec.wv.index2entity[:vocab_count]:
        # print(word, w2vec.wv.vocab[word].count)
        writer.write(word + ' ' + str(w2vec.wv.vocab[word].count) + '\n') # Output vocab count
print("Finished writing vocab file")

w2vec.wv.save_word2vec_format('word2Vec/gensim_%s_%s.300d.txt'%(big_categories,small_categories), binary=False)

Writing vocab file...
Finished writing vocab file


2019-12-11 00:31:48,089 : INFO : storing 14583x300 projection weights into word2Vec/gensim_Electronics_DVD Players.300d.txt
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
w2vec.corpus_count
w2vec.vector_size

300

In [16]:
w2vec.most_similar(u"player", topn=10)

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-12-11 00:31:52,189 : INFO : precomputing L2-norms of word weight vectors
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('recorder', 0.7920495271682739),
 ('changer', 0.7253953218460083),
 ('unit', 0.7008682489395142),
 ('machine', 0.6885756254196167),
 ('model', 0.593877911567688),
 ('players', 0.582635760307312),
 ('modification', 0.5821385979652405),
 ('deck', 0.5558204650878906),
 ('device', 0.5466880202293396),
 ('product', 0.5403242111206055)]

In [17]:
w2vec.most_similar(['dvd','player','changer','machine','video'], topn=20)

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('recorder', 0.7480524778366089),
 ('deck', 0.6619045734405518),
 ('unit', 0.6347320079803467),
 ('device', 0.6124114990234375),
 ('model', 0.6094457507133484),
 ('rebate', 0.608971893787384),
 ('sd', 0.6077044606208801),
 ('exc', 0.5832141041755676),
 ('cdrmpjeg', 0.582229495048523),
 ('writes', 0.5762830376625061),
 ('dvp', 0.5707143545150757),
 ('carousel', 0.5653432607650757),
 ('tablet', 0.5644240379333496),
 ('format', 0.5600840449333191),
 ('modification', 0.5533640384674072),
 ('vcr', 0.5526679158210754),
 ('hunt', 0.5517515540122986),
 ('kenwood', 0.5361608266830444),
 ('noiseless', 0.5309823751449585),
 ('otherhand', 0.5243709683418274)]

In [18]:
from sklearn.manifold import TSNE
import re
import matplotlib.pyplot as plt
# vocab = list(w2vec.vocabulary)

vocab = {word: w2vec.wv.vocab[word].count for word in w2vec.wv.index2entity}
# vocab
# X = list(vocab.values())

# tsne = TSNE(n_components=2)
# X_tsne = tsne.fit_transform(X)
# df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y'])

In [19]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos)
    
plt.show()

NameError: name 'df' is not defined